# 股票成交量变化预测

In [1]:
from pandas_datareader.data import DataReader

import torch
import torch.nn
import torch.optim

数据读取

In [2]:
df = DataReader('FB.US', 'quandl', '2012-01-01', '2018-02-01')
df

,Open,High,Low,Close,Volume,ExDividend,SplitRatio,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjVolume
Date,,,,,,,,,,,,
2018-02-01,188.220,195.3200,187.8900,193.0900,53608910.0,0.0,1.0,188.220,195.3200,187.8900,193.0900,53608910.0
2018-01-31,188.370,189.8300,185.2200,186.8900,27299152.0,0.0,1.0,188.370,189.8300,185.2200,186.8900,27299152.0
2018-01-30,183.010,188.1800,181.8400,187.1200,20382841.0,0.0,1.0,183.010,188.1800,181.8400,187.1200,20382841.0
2018-01-29,188.750,188.8400,185.6301,185.9800,20022931.0,0.0,1.0,188.750,188.8400,185.6301,185.9800,20022931.0
2018-01-26,187.750,190.0000,186.8100,190.0000,16652197.0,0.0,1.0,187.750,190.0000,186.8100,190.0000,16652197.0
2018-01-25,187.950,188.6200,186.6000,187.4800,16698537.0,0.0,1.0,187.950,188.6200,186.6000,187.4800,16698537.0
2018-01-24,189.890,190.6600,186.5200,186.5500,22992031.0,0.0,1.0,189.890,190.6600,186.5200,186.5500,22992031.0
2018-01-23,186.050,189.5500,185.5500,189.3500,24956444.0,0.0,1.0,186.050,189.5500,185.5500,189.3500,24956444.0
2018-01-22,180.800,185.3900,180.4100,185.3700,20567285.0,0.0,1.0,180.800,185.3900,180.4100,185.3700,20567285.0


In [3]:
train_start, train_end = sum(df.index >= '2017'), sum(df.index >= '2013')
test_start, test_end = sum(df.index >= '2018'), sum(df.index >= '2017')
n_total_train = train_end - train_start
n_total_test = test_end - test_start
s_mean = df[train_start:train_end].mean()
s_std = df[train_start:train_end].std()
n_features = 5
df_feature = ((df - s_mean) / s_std).iloc[:, :n_features]
s_label = (df['Volume'] < df['Volume'].shift(1)).astype(int)
df_feature, s_label

(                Open      High       Low     Close     Volume
 Date                                                         
 2018-02-01  3.549770  3.728652  3.595515  3.707593   0.468477
 2018-01-31  3.554580  3.553594  3.509259  3.508705  -0.438438
 2018-01-30  3.382700  3.500981  3.400065  3.516083  -0.676848
 2018-01-29  3.566766  3.522026  3.522507  3.479513  -0.689255
 2018-01-26  3.534699  3.559014  3.560625  3.608470  -0.805446
 2018-01-25  3.541112  3.515011  3.553840  3.527631  -0.803849
 2018-01-24  3.603323  3.580060  3.551256  3.497798  -0.586908
 2018-01-23  3.480184  3.544665  3.519919  3.587619  -0.519193
 2018-01-22  3.311831  3.412017  3.353868  3.459945  -0.670491
 2018-01-19  3.313434  3.315719  3.346121  3.329063  -0.474049
 2018-01-18  3.226211  3.271396  3.246290  3.281266  -0.594087
 2018-01-17  3.262447  3.218464  3.204939  3.210692  -0.436445
 2018-01-16  3.334278  3.295949  3.277304  3.236034  -0.172048
 2018-01-12  3.223967  3.287339  3.256628  3.267472   1

训练和测试

In [4]:
fc = torch.nn.Linear(n_features, 1)
weights, bias = fc.parameters()
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(fc.parameters())

x = torch.tensor(df_feature.values, dtype=torch.float32)
y = torch.tensor(s_label.values.reshape(-1, 1), dtype=torch.float32)

n_step = 20001
for step in range(n_step):
    if step:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    pred = fc(x)
    loss = criterion(pred[train_start:train_end], y[train_start:train_end])
    
    if step % 500 == 0:
        # print('#{}, 损失 = {:g}, 权重 weights = {}, bias = {:g}'.format(
        #         step, loss, weights[0, :].tolist(), bias.item()))
        print('#{}, 损失 = {:g}, '.format(step, loss))
        
        output = (pred > 0)
        correct = (output == y.byte())
        n_correct_train = correct[train_start:train_end].sum().item()
        n_correct_test = correct[test_start:test_end].sum().item()
        accuracy_train = n_correct_train / n_total_train
        accuracy_test = n_correct_test / n_total_test
        print('训练集准确率 = {}, 测试集准确率 = {}'.format(accuracy_train, accuracy_test))

#0, 损失 = 0.71349, 
训练集准确率 = 0.47023809523809523, 测试集准确率 = 0.456
#500, 损失 = 0.665129, 
训练集准确率 = 0.5912698412698413, 测试集准确率 = 0.544
#1000, 损失 = 0.659458, 
训练集准确率 = 0.6150793650793651, 测试集准确率 = 0.544
#1500, 损失 = 0.658672, 
训练集准确率 = 0.6130952380952381, 测试集准确率 = 0.548
#2000, 损失 = 0.658524, 
训练集准确率 = 0.6101190476190477, 测试集准确率 = 0.548
#2500, 损失 = 0.658388, 
训练集准确率 = 0.6081349206349206, 测试集准确率 = 0.548
#3000, 损失 = 0.658215, 
训练集准确率 = 0.6101190476190477, 测试集准确率 = 0.548
#3500, 损失 = 0.658, 
训练集准确率 = 0.6130952380952381, 测试集准确率 = 0.544
#4000, 损失 = 0.657739, 
训练集准确率 = 0.6111111111111112, 测试集准确率 = 0.544
#4500, 损失 = 0.657428, 
训练集准确率 = 0.6121031746031746, 测试集准确率 = 0.544
#5000, 损失 = 0.657068, 
训练集准确率 = 0.6140873015873016, 测试集准确率 = 0.552
#5500, 损失 = 0.656664, 
训练集准确率 = 0.6150793650793651, 测试集准确率 = 0.556
#6000, 损失 = 0.656228, 
训练集准确率 = 0.6150793650793651, 测试集准确率 = 0.548
#6500, 损失 = 0.65577, 
训练集准确率 = 0.6160714285714286, 测试集准确率 = 0.556
#7000, 损失 = 0.655305, 
训练集准确率 = 0.6180555555555556, 测试集准确率 = 0.564
#75